In [1]:
#This entire notebook shouldn't take more than 2 hours to run
#run this notebook after running 
import requests
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import numpy as npy
import re
from datetime import datetime, timedelta, date
import random

In [2]:
#updated scraped fight data (after running fight_hist_updated function from UFC_data_scraping file)
fight_hist=pd.read_csv('fight_hist.csv',sep=',')
#all stats fight history file which is one update behind fight_hist
ufcfightscrap=pd.read_csv('ufc_fights_crap.csv',sep=',',low_memory=False)
#updated scraped fighter data (after running fight_hist_updated function from UFC_data_scraping file)
ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')
#most recent fight in fight_hist versus most recent fight in ufcfightscrap

In [3]:
#this cell contains all functions defined for building columns in ufcfightscrap
#converts from '%B %d, %Y' (i.e. August 22, 2020) to date (i.e. 2020-08-22)
def convert_to_datetime(day1):
    return datetime.strptime(day1, '%B %d, %Y').date()
convert_to_datetime_vect= npy.vectorize(convert_to_datetime)

#this age function is written in such a stupid way
def age(birthDate,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    if birthDate=='--':
        aa='unknown'
    elif type(birthDate)==str and not type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif type(birthDate)==str and type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif not type(birthDate)==str and type(day)==str:
        bd=birthDate
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    else:
        bd=birthDate
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    return float(aa)

age_vect= npy.vectorize(age)

#zero function (needed for reversals)
def zero(x):
    return 0

zero_vect= npy.vectorize(zero)

def fighter_age(fighter,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            dob=datetime.strptime(ufcfighterscrap['dob'][i], '%b %d, %Y').strftime('%B %d, %Y')
            a=age(dob,day,form1,form2)
            break
    return float(a)

fighter_age_vect= npy.vectorize(fighter_age)

def wins_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
    return float(summ)

def losses_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
    return float(summ)

#functions for height and reach
def fighter_height(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['height'][i]
            break
    if a=='--' or type(a)==int:
        b='unknown'
    elif a[4]=='"':
        b=int(a[0])*30.48+int(a[3])*2.54
    else:
        b=int(a[0])*30.48+int(a[3]+a[4])*2.54
    try:
        return float(b)
    except:
        return b

def fighter_reach(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['reach'][i]
            break
    if a=='--' or type(a)==int:
        b='unknown'
    else:
        b=int(a[0]+a[1])*2.54
    try:
        return float(b)
    except:
        return b

wins_before_vect= npy.vectorize(wins_before)
losses_before_vect= npy.vectorize(losses_before)
fighter_height_vect= npy.vectorize(fighter_height)
fighter_reach_vect= npy.vectorize(fighter_reach)

#day1 should be input in the form '%B %d, %Y' i.e. 'August 20, 1962'
#conversions can be made via day=datetime.strptime(ufcfightsML_known_df['date'][i], '%B %d, %Y').strftime('%b %d, %Y')
def time_diff(day1,day2=date.today()):
    if day2==date.today():
        answer=(day2-datetime.strptime(day, '%B %d, %Y')).days
    else:
        answer=(datetime.strptime(day2, '%B %d, %Y')-datetime.strptime(day1, '%B %d, %Y')).days
    return answer

#we now vectorize this to use in pandas/numpy
time_diff_vect= npy.vectorize(time_diff)

#can make a single function to do all of these... actually maybe the count function would even work as is

def L5Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return float(summ)

def ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return float(summ)

def L5Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return float(summ)

def sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return float(summ)

def L5Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

#vectorize all of these functions
L5Y_wins_vect= npy.vectorize(L5Y_wins)
L5Y_losses_vect= npy.vectorize(L5Y_losses)
L2Y_wins_vect= npy.vectorize(L2Y_wins)
L2Y_losses_vect= npy.vectorize(L2Y_losses)
ko_wins_vect= npy.vectorize(ko_wins)
ko_losses_vect= npy.vectorize(ko_losses)
L5Y_ko_wins_vect= npy.vectorize(L5Y_ko_wins)
L5Y_ko_losses_vect= npy.vectorize(L5Y_ko_losses)
L2Y_ko_wins_vect= npy.vectorize(L2Y_ko_wins)
L2Y_ko_losses_vect= npy.vectorize(L2Y_ko_losses)
sub_wins_vect= npy.vectorize(sub_wins)
sub_losses_vect= npy.vectorize(sub_losses)
L5Y_sub_wins_vect= npy.vectorize(L5Y_sub_wins)
L5Y_sub_losses_vect= npy.vectorize(L5Y_sub_losses)
L2Y_sub_wins_vect= npy.vectorize(L2Y_sub_wins)
L2Y_sub_losses_vect= npy.vectorize(L2Y_sub_losses)

#for columns like fighter_rec which contains the information for the opponent as well, we use the following
def opponent_column(stat):
    col=dict()
    for i in range(len(ufcfightscrap['fighter'])):
        if i%2==0:
            col[i]=ufcfightscrap[stat][i+1]
        else:
            col[i]=ufcfightscrap[stat][i-1]
    statdict={'stat':col}
    return pd.DataFrame (statdict, columns = ['stat'])

#enter date unabbreviated 'July 4, 2019'
#here the average gives avg per fight. Later in avg_count we change to average per time spent in octagon
def count(stat, guy,inf_abs, total_L5Y_L2Y_avg, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if total_L5Y_L2Y_avg=='total' or total_L5Y_L2Y_avg=='avg':
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0]
    elif total_L5Y_L2Y_avg=='L2Y':
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<730]
    else:
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<1825]
    if inf_abs=='inf':
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    else:
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy]
    good_indices=[i for i in good_indices_1 if i in good_indices_2]
    if total_L5Y_L2Y_avg!='avg':
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
    else:
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
        day1=convert_date_to_abbrev(day1)
        number_fights=wins_before(guy,day1)+losses_before(guy,day1)
        summ=summ/float(number_fights)
    return summ
        
#note a better average is per time not per fight. We will do each stat as an average per one minutes

#enter time in the form 'August 24, 2018'
def time_in_octagon(guy,day1=date.today().strftime('%B %d, %Y')):
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['fighter'][i]==guy] 
    for i in good_indices:
        if ufcfightscrap['time'][i][2]==':':
            summ=int(ufcfightscrap['time'][i][0:2])+int(ufcfightscrap['time'][i][3:])/60.0
        else:
            summ+=5*(ufcfightscrap['round'][i]-1)+int(ufcfightscrap['time'][i][0])+int(ufcfightscrap['time'][i][2:])/60.0
    return summ

#enter date unabbreviated 'July 4, 2019'
#gives takedowns per minute
def avg_count(stat, guy,inf_abs, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if inf_abs=='inf':
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    else:
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    for i in good_indices:
        summ+=ufcfightscrap[stat][i]
    t= time_in_octagon(guy,day1)
    if t==0:
        summ=0
    else:
        summ=summ/t
    return summ

#vectorize these functions
count_vect= npy.vectorize(count)
avg_count_vect= npy.vectorize(avg_count)
    


def stance(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['stance'][i]
            break
    if a=='Orthodox':
        return 0
    elif a=='Switch':
        return 1
    elif a=='Southpaw':
        return 2
    elif a=='Open Stance':
        return 3
    elif a=='Sideways':
        return 4
    else:
        return 5
    
stance_vect= npy.vectorize(stance)

In [4]:
update_time=time_diff(ufcfightscrap['date'][0],fight_hist['date'][0])
print(update_time)

7


In [5]:
#this gives the new rows in fight_hist which do not appear in ufcfightscrapd
new_rows=fight_hist.loc[time_diff_vect(fight_hist['date'],fight_hist['date'][0])<update_time]
#convert to numpy array
numpy_new_rows=new_rows.values
#gives new_rows as a dataframe instead of a clone
test_new_rows=pd.DataFrame(data=numpy_new_rows,    # 1st column as index
                columns=new_rows.columns)  # 1st row as the column names

In [6]:
#adding a datetime column for test_new_rows
#test_new_rows['datetime']=convert_to_datetime_vect(test_new_rows['date'])
#setting up columns for rec,wins,losses,ties (can be done in one line each as we do later with other stats)
#run time for each is 20 min per pair
test_new_rows['fighter_wins']=wins_before_vect(test_new_rows['fighter'],test_new_rows['date'])
#column for fighter losses in test_new_rows
test_new_rows['fighter_losses']=losses_before_vect(test_new_rows['fighter'],test_new_rows['date'])
#column for fighter age in test_new_rows
test_new_rows['fighter_age']=fighter_age_vect(test_new_rows['fighter'],test_new_rows['date'])
#column for fighter height in test_new_rows
test_new_rows['fighter_height']=test_new_rows['fighter'].apply(fighter_height)
#column for fighter reach in test_new_rows
test_new_rows['fighter_reach']=test_new_rows['fighter'].apply(fighter_height)

In [7]:
test_new_rows

,date,fight_url,event_url,result,fighter,opponent,division,method,round,time,...,distance_strikes_attempts,clinch_strikes_landed,clinch_strikes_attempts,ground_strikes_landed,ground_strikes_attempts,fighter_wins,fighter_losses,fighter_age,fighter_height,fighter_reach
0,"March 12, 2022",http://ufcstats.com/fight-details/8aa16cc1ba99...,http://ufcstats.com/event-details/1f9344211ca7...,W,Magomed Ankalaev,Thiago Santos,Light Heavyweight,U-DEC,5,5:00,...,140,12,13,2,2,7.0,1.0,29.0,190.50,190.50
1,"March 12, 2022",http://ufcstats.com/fight-details/8aa16cc1ba99...,http://ufcstats.com/event-details/1f9344211ca7...,L,Thiago Santos,Magomed Ankalaev,Light Heavyweight,U-DEC,5,5:00,...,135,1,1,2,2,14.0,8.0,38.0,187.96,187.96
2,"March 12, 2022",http://ufcstats.com/fight-details/9b46a662b8fa...,http://ufcstats.com/event-details/1f9344211ca7...,W,Song Yadong,Marlon Moraes,Bantamweight,KO/TKO,1,2:06,...,35,0,0,0,0,7.0,1.0,24.0,172.72,172.72
3,"March 12, 2022",http://ufcstats.com/fight-details/9b46a662b8fa...,http://ufcstats.com/event-details/1f9344211ca7...,L,Marlon Moraes,Song Yadong,Bantamweight,KO/TKO,1,2:06,...,25,0,0,0,0,5.0,5.0,33.0,167.64,167.64
4,"March 12, 2022",http://ufcstats.com/fight-details/70b8d357e893...,http://ufcstats.com/event-details/1f9344211ca7...,W,Sodiq Yusuff,Alex Caceres,Featherweight,U-DEC,3,5:00,...,117,5,6,0,0,4.0,1.0,28.0,175.26,175.26
5,"March 12, 2022",http://ufcstats.com/fight-details/70b8d357e893...,http://ufcstats.com/event-details/1f9344211ca7...,L,Alex Caceres,Sodiq Yusuff,Featherweight,U-DEC,3,5:00,...,133,2,5,0,0,14.0,10.0,33.0,177.80,177.80
6,"March 12, 2022",http://ufcstats.com/fight-details/23eec2d9cb2d...,http://ufcstats.com/event-details/1f9344211ca7...,W,Khalil Rountree Jr.,Karl Roberson,Light Heavyweight,KO/TKO,2,0:25,...,55,5,7,4,7,5.0,5.0,32.0,185.42,185.42
7,"March 12, 2022",http://ufcstats.com/fight-details/23eec2d9cb2d...,http://ufcstats.com/event-details/1f9344211ca7...,L,Karl Roberson,Khalil Rountree Jr.,Light Heavyweight,KO/TKO,2,0:25,...,27,1,1,0,0,4.0,4.0,31.0,185.42,185.42
8,"March 12, 2022",http://ufcstats.com/fight-details/463fac0e4850...,http://ufcstats.com/event-details/1f9344211ca7...,W,Drew Dober,Terrance McKinney,Lightweight,KO/TKO,1,3:17,...,19,2,2,9,16,9.0,7.0,33.0,172.72,172.72
9,"March 12, 2022",http://ufcstats.com/fight-details/463fac0e4850...,http://ufcstats.com/event-details/1f9344211ca7...,L,Terrance McKinney,Drew Dober,Lightweight,KO/TKO,1,3:17,...,32,5,5,16,24,2.0,0.0,27.0,177.80,177.80


In [8]:
#making columns for fighter_L5Y_wins losses etc (about 5 minutes or less)
test_new_rows['fighter_L5Y_wins']=L5Y_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_losses']=L5Y_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_wins']=L2Y_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_losses']=L2Y_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_ko_wins']=ko_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_ko_losses']=ko_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_ko_wins']=L5Y_ko_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_ko_losses']=L5Y_ko_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_ko_wins']=L2Y_ko_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_ko_losses']=L2Y_ko_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_sub_wins']=sub_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_sub_losses']=sub_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_sub_wins']=L5Y_sub_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_sub_losses']=L5Y_sub_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_sub_wins']=L2Y_sub_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_sub_losses']=L2Y_sub_losses_vect(test_new_rows['fighter'],test_new_rows['date'])

In [9]:
time test_row = avg_count_vect('knockdowns',test_new_rows['fighter'],'inf',test_new_rows['date'])


CPU times: user 5.28 s, sys: 29.7 ms, total: 5.31 s
Wall time: 5.31 s


In [10]:
test_new_rows['fighter_inf_knockdowns_avg']=test_row;
test_new_rows

,date,fight_url,event_url,result,fighter,opponent,division,method,round,time,...,fighter_L5Y_ko_losses,fighter_L2Y_ko_wins,fighter_L2Y_ko_losses,fighter_sub_wins,fighter_sub_losses,fighter_L5Y_sub_wins,fighter_L5Y_sub_losses,fighter_L2Y_sub_wins,fighter_L2Y_sub_losses,fighter_inf_knockdowns_avg
0,"March 12, 2022",http://ufcstats.com/fight-details/8aa16cc1ba99...,http://ufcstats.com/event-details/1f9344211ca7...,W,Magomed Ankalaev,Thiago Santos,Light Heavyweight,U-DEC,5,5:00,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.076368
1,"March 12, 2022",http://ufcstats.com/fight-details/8aa16cc1ba99...,http://ufcstats.com/event-details/1f9344211ca7...,L,Thiago Santos,Magomed Ankalaev,Light Heavyweight,U-DEC,5,5:00,...,1.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0,0.066214
2,"March 12, 2022",http://ufcstats.com/fight-details/9b46a662b8fa...,http://ufcstats.com/event-details/1f9344211ca7...,W,Song Yadong,Marlon Moraes,Bantamweight,KO/TKO,1,2:06,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.040858
3,"March 12, 2022",http://ufcstats.com/fight-details/9b46a662b8fa...,http://ufcstats.com/event-details/1f9344211ca7...,L,Marlon Moraes,Song Yadong,Bantamweight,KO/TKO,1,2:06,...,4.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.059488
4,"March 12, 2022",http://ufcstats.com/fight-details/70b8d357e893...,http://ufcstats.com/event-details/1f9344211ca7...,W,Sodiq Yusuff,Alex Caceres,Featherweight,U-DEC,3,5:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038860
5,"March 12, 2022",http://ufcstats.com/fight-details/70b8d357e893...,http://ufcstats.com/event-details/1f9344211ca7...,L,Alex Caceres,Sodiq Yusuff,Featherweight,U-DEC,3,5:00,...,0.0,0.0,0.0,4.0,5.0,2.0,1.0,2.0,0.0,0.013436
6,"March 12, 2022",http://ufcstats.com/fight-details/23eec2d9cb2d...,http://ufcstats.com/event-details/1f9344211ca7...,W,Khalil Rountree Jr.,Karl Roberson,Light Heavyweight,KO/TKO,2,0:25,...,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.107720
7,"March 12, 2022",http://ufcstats.com/fight-details/23eec2d9cb2d...,http://ufcstats.com/event-details/1f9344211ca7...,L,Karl Roberson,Khalil Rountree Jr.,Light Heavyweight,KO/TKO,2,0:25,...,0.0,0.0,0.0,2.0,4.0,2.0,4.0,0.0,2.0,0.015385
8,"March 12, 2022",http://ufcstats.com/fight-details/463fac0e4850...,http://ufcstats.com/event-details/1f9344211ca7...,W,Drew Dober,Terrance McKinney,Lightweight,KO/TKO,1,3:17,...,0.0,1.0,0.0,1.0,4.0,0.0,2.0,0.0,1.0,0.034180
9,"March 12, 2022",http://ufcstats.com/fight-details/463fac0e4850...,http://ufcstats.com/event-details/1f9344211ca7...,L,Terrance McKinney,Drew Dober,Lightweight,KO/TKO,1,3:17,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.434783


In [11]:
#forming columns for fighter_inf_knockdowns_avg etc... for (1500 new examples takes 90 minutes)
test_new_rows['fighter_inf_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_pass_avg']=avg_count_vect('pass',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_reversals_avg']=zero_vect(test_new_rows['fighter'])
test_new_rows['fighter_inf_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
print('quarter done')
test_new_rows['fighter_inf_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
print('half done')
test_new_rows['fighter_inf_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
print('almost done')
test_new_rows['fighter_inf_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])


quarter done
half done
almost done


In [12]:
#forming columns for fighter_abs_knockdowns_avg etc...
test_new_rows['fighter_abs_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_pass_avg']=avg_count_vect('pass',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_reversals_avg']=zero_vect(test_new_rows['fighter'])
test_new_rows['fighter_abs_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
print('quarter done')
test_new_rows['fighter_abs_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
print('half done')
test_new_rows['fighter_abs_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
print('almost done')
test_new_rows['fighter_abs_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])

quarter done
half done
almost done


In [13]:
#opponent columns for wins etc
test_new_rows['opponent_wins']=opponent_column('fighter_wins')
test_new_rows['opponent_losses']=opponent_column('fighter_losses')
test_new_rows['opponent_age']=opponent_column('fighter_age')
test_new_rows['opponent_height']=opponent_column('fighter_height')
test_new_rows['opponent_reach']=opponent_column('fighter_reach')
#making columns for opponent_L5Y_wins losses etc
print('quarter done')
test_new_rows['opponent_L5Y_wins']=opponent_column('fighter_L5Y_wins')
test_new_rows['opponent_L5Y_losses']=opponent_column('fighter_L5Y_losses')
test_new_rows['opponent_L2Y_wins']=opponent_column('fighter_L2Y_wins')
test_new_rows['opponent_L2Y_losses']=opponent_column('fighter_L2Y_losses')
test_new_rows['opponent_ko_wins']=opponent_column('fighter_ko_wins')
test_new_rows['opponent_ko_losses']=opponent_column('fighter_ko_losses')
print('half done')
test_new_rows['opponent_L5Y_ko_wins']=opponent_column('fighter_L5Y_ko_wins')
test_new_rows['opponent_L5Y_ko_losses']=opponent_column('fighter_L5Y_ko_losses')
test_new_rows['opponent_L2Y_ko_wins']=opponent_column('fighter_L2Y_ko_wins')
test_new_rows['opponent_L2Y_ko_losses']=opponent_column('fighter_L2Y_ko_losses')
test_new_rows['opponent_sub_wins']=opponent_column('fighter_sub_wins')
print('almost done')
test_new_rows['opponent_sub_losses']=opponent_column('fighter_sub_losses')
test_new_rows['opponent_L5Y_sub_wins']=opponent_column('fighter_L5Y_sub_wins')
test_new_rows['opponent_L5Y_sub_losses']=opponent_column('fighter_L5Y_sub_losses')
test_new_rows['opponent_L2Y_sub_wins']=opponent_column('fighter_L2Y_sub_wins')
test_new_rows['opponent_L2Y_sub_losses']=opponent_column('fighter_L2Y_sub_losses')

quarter done
half done
almost done


In [14]:
#forming columns for opponent_inf_knockdowns_avg etc...
test_new_rows['opponent_inf_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_pass_avg']=avg_count_vect('pass',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_reversals_avg']=zero_vect(test_new_rows['opponent'])
test_new_rows['opponent_inf_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
print('quarter done')
test_new_rows['opponent_inf_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
print('half done')
test_new_rows['opponent_inf_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
print('almost done')
test_new_rows['opponent_inf_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])

quarter done
half done


/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/2569836320.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/2569836320.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_inf_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['opponent'],'inf',test_

almost done


/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/2569836320.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_inf_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/2569836320.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_inf_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['opponent

In [15]:
#forming columns for opponent_abs_knockdowns_avg etc...
test_new_rows['opponent_abs_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_pass_avg']=avg_count_vect('pass',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_reversals_avg']=zero_vect(test_new_rows['opponent'])
test_new_rows['opponent_abs_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
print('quarter done')
test_new_rows['opponent_abs_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
print('half done')
test_new_rows['opponent_abs_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
print('almost done')
test_new_rows['opponent_abs_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])

/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/290036775.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['opponent'],'abs',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/290036775.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_pass_avg']=avg_count_vect('pass',test_new_rows['opponent'],'abs',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t

quarter done


/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/290036775.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/290036775.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['opponent'],'abs',test_

half done


/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/290036775.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/290036775.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['opponent'],'abs',test_ne

almost done


/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/290036775.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/290036775.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['opponent']

In [16]:
test_new_rows['fighter_stance']=stance_vect(test_new_rows['fighter'])
test_new_rows['opponent_stance']=stance_vect(test_new_rows['opponent'])

/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/86544192.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['fighter_stance']=stance_vect(test_new_rows['fighter'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_95296/86544192.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_stance']=stance_vect(test_new_rows['opponent'])


In [17]:
test_new_rows

,date,fight_url,event_url,result,fighter,opponent,division,method,round,time,...,opponent_abs_leg_strikes_landed_avg,opponent_abs_leg_strikes_attempts_avg,opponent_abs_distance_strikes_landed_avg,opponent_abs_distance_strikes_attempts_avg,opponent_abs_clinch_strikes_landed_avg,opponent_abs_clinch_strikes_attempts_avg,opponent_abs_ground_strikes_landed_avg,opponent_abs_ground_strikes_attempts_avg,fighter_stance,opponent_stance
0,"March 12, 2022",http://ufcstats.com/fight-details/8aa16cc1ba99...,http://ufcstats.com/event-details/1f9344211ca7...,W,Magomed Ankalaev,Thiago Santos,Light Heavyweight,U-DEC,5,5:00,...,0.631579,0.942275,1.533107,4.268251,0.285229,0.387097,0.458404,0.570458,0,0
1,"March 12, 2022",http://ufcstats.com/fight-details/8aa16cc1ba99...,http://ufcstats.com/event-details/1f9344211ca7...,L,Thiago Santos,Magomed Ankalaev,Light Heavyweight,U-DEC,5,5:00,...,0.610946,0.687314,1.756470,5.040305,0.292745,0.394569,0.000000,0.025456,0,0
2,"March 12, 2022",http://ufcstats.com/fight-details/9b46a662b8fa...,http://ufcstats.com/event-details/1f9344211ca7...,W,Song Yadong,Marlon Moraes,Bantamweight,KO/TKO,1,2:06,...,0.523498,0.701963,2.974420,7.816776,0.285544,0.356930,1.213563,1.534801,0,0
3,"March 12, 2022",http://ufcstats.com/fight-details/9b46a662b8fa...,http://ufcstats.com/event-details/1f9344211ca7...,L,Marlon Moraes,Song Yadong,Bantamweight,KO/TKO,1,2:06,...,0.572012,0.817160,3.207354,8.600613,0.296221,0.418795,0.265577,0.347293,0,0
4,"March 12, 2022",http://ufcstats.com/fight-details/70b8d357e893...,http://ufcstats.com/event-details/1f9344211ca7...,W,Sodiq Yusuff,Alex Caceres,Featherweight,U-DEC,3,5:00,...,0.621431,0.796103,2.361438,7.396708,0.201545,0.302318,0.285522,0.473631,0,2
5,"March 12, 2022",http://ufcstats.com/fight-details/70b8d357e893...,http://ufcstats.com/event-details/1f9344211ca7...,L,Alex Caceres,Sodiq Yusuff,Featherweight,U-DEC,3,5:00,...,0.388601,0.446891,2.545337,7.247409,0.408031,0.544041,0.019430,0.077720,2,0
6,"March 12, 2022",http://ufcstats.com/fight-details/23eec2d9cb2d...,http://ufcstats.com/event-details/1f9344211ca7...,W,Khalil Rountree Jr.,Karl Roberson,Light Heavyweight,KO/TKO,2,0:25,...,0.246154,0.446154,1.230769,4.107692,0.138462,0.169231,1.000000,1.276923,2,2
7,"March 12, 2022",http://ufcstats.com/fight-details/23eec2d9cb2d...,http://ufcstats.com/event-details/1f9344211ca7...,L,Karl Roberson,Khalil Rountree Jr.,Light Heavyweight,KO/TKO,2,0:25,...,0.873728,1.077199,2.956314,6.295631,0.119689,0.191502,0.502693,0.622382,2,2
8,"March 12, 2022",http://ufcstats.com/fight-details/463fac0e4850...,http://ufcstats.com/event-details/1f9344211ca7...,W,Drew Dober,Terrance McKinney,Lightweight,KO/TKO,1,3:17,...,0.434783,0.434783,0.434783,0.434783,0.000000,0.000000,0.000000,0.000000,2,1
9,"March 12, 2022",http://ufcstats.com/fight-details/463fac0e4850...,http://ufcstats.com/event-details/1f9344211ca7...,L,Terrance McKinney,Drew Dober,Lightweight,KO/TKO,1,3:17,...,0.334966,0.410163,3.301812,7.540162,0.499032,0.704113,0.280278,0.382819,1,2


In [18]:
#joining test_new_rows to ufcfightscrap to get the most updated version
frames = [test_new_rows, ufcfightscrap]
updated_ufcfightscrap = pd.concat(frames,ignore_index=True)

In [19]:
updated_ufcfightscrap

,date,fight_url,event_url,result,fighter,opponent,division,method,round,time,...,opponent_abs_leg_strikes_landed_avg,opponent_abs_leg_strikes_attempts_avg,opponent_abs_distance_strikes_landed_avg,opponent_abs_distance_strikes_attempts_avg,opponent_abs_clinch_strikes_landed_avg,opponent_abs_clinch_strikes_attempts_avg,opponent_abs_ground_strikes_landed_avg,opponent_abs_ground_strikes_attempts_avg,fighter_stance,opponent_stance
0,"March 12, 2022",http://ufcstats.com/fight-details/8aa16cc1ba99...,http://ufcstats.com/event-details/1f9344211ca7...,W,Magomed Ankalaev,Thiago Santos,Light Heavyweight,U-DEC,5,5:00,...,0.631579,0.942275,1.533107,4.268251,0.285229,0.387097,0.458404,0.570458,0,0
1,"March 12, 2022",http://ufcstats.com/fight-details/8aa16cc1ba99...,http://ufcstats.com/event-details/1f9344211ca7...,L,Thiago Santos,Magomed Ankalaev,Light Heavyweight,U-DEC,5,5:00,...,0.610946,0.687314,1.756470,5.040305,0.292745,0.394569,0.000000,0.025456,0,0
2,"March 12, 2022",http://ufcstats.com/fight-details/9b46a662b8fa...,http://ufcstats.com/event-details/1f9344211ca7...,W,Song Yadong,Marlon Moraes,Bantamweight,KO/TKO,1,2:06,...,0.523498,0.701963,2.974420,7.816776,0.285544,0.356930,1.213563,1.534801,0,0
3,"March 12, 2022",http://ufcstats.com/fight-details/9b46a662b8fa...,http://ufcstats.com/event-details/1f9344211ca7...,L,Marlon Moraes,Song Yadong,Bantamweight,KO/TKO,1,2:06,...,0.572012,0.817160,3.207354,8.600613,0.296221,0.418795,0.265577,0.347293,0,0
4,"March 12, 2022",http://ufcstats.com/fight-details/70b8d357e893...,http://ufcstats.com/event-details/1f9344211ca7...,W,Sodiq Yusuff,Alex Caceres,Featherweight,U-DEC,3,5:00,...,0.621431,0.796103,2.361438,7.396708,0.201545,0.302318,0.285522,0.473631,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13017,"November 12, 1993",http://ufcstats.com/fight-details/cecdc0da5842...,http://ufcstats.com/event-details/6420efac0578...,L,Art Jimmerson,Royce Gracie,Open Weight,SUB,1,2:18,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2
13018,"November 12, 1993",http://ufcstats.com/fight-details/2d2bbc86e941...,http://ufcstats.com/event-details/6420efac0578...,W,Kevin Rosier,Zane Frazier,Open Weight,KO/TKO,1,4:20,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
13019,"November 12, 1993",http://ufcstats.com/fight-details/2d2bbc86e941...,http://ufcstats.com/event-details/6420efac0578...,L,Zane Frazier,Kevin Rosier,Open Weight,KO/TKO,1,4:20,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
13020,"November 12, 1993",http://ufcstats.com/fight-details/567a09fd200c...,http://ufcstats.com/event-details/6420efac0578...,W,Gerard Gordeau,Teila Tuli,Open Weight,KO/TKO,1,0:26,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0


In [20]:
#saving the updated ufcfightscrap file
updated_ufcfightscrap.to_csv('ufc_fights_crap.csv', index = False)

In [21]:
#here is the list of all stats available (besides stance), does not include names or result
computed_statistics=[u'fighter_wins', u'fighter_losses', u'fighter_age',u'fighter_height',
                    u'fighter_reach', u'fighter_L5Y_wins',u'fighter_L5Y_losses', u'fighter_L2Y_wins', u'fighter_L2Y_losses',
                    u'fighter_ko_wins',u'fighter_ko_losses',u'fighter_L5Y_ko_wins',u'fighter_L5Y_ko_losses',u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',u'fighter_sub_wins',u'fighter_sub_losses',u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', u'fighter_L2Y_sub_wins', u'fighter_L2Y_sub_losses',
                    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    u'opponent_wins', u'opponent_losses', u'opponent_age',  u'opponent_height',
                    u'opponent_reach',   u'opponent_L5Y_wins', u'opponent_L5Y_losses', u'opponent_L2Y_wins', u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', u'opponent_ko_losses', u'opponent_L5Y_ko_wins', u'opponent_L5Y_ko_losses', u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', u'opponent_sub_wins', u'opponent_sub_losses',u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', u'opponent_L2Y_sub_wins', u'opponent_L2Y_sub_losses',
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg',
                    u'fighter_stance',
                    u'opponent_stance']



In [22]:
#list containing all columns of any interest
relevant_list=['date','division','fighter','opponent','result','method']
relevant_list.extend(computed_statistics)

In [23]:
#creates a clean file with only columns which are relevant to predicting
updated_ufc_fights=updated_ufcfightscrap[relevant_list]

In [24]:
#lets randomly remove one of every two copied fights
random_indices=[]
for i in range(0,len(updated_ufc_fights['fighter_wins']),2):
    random_indices.append(random.choice([i,i+1]))
    
updated_ufc_fights=updated_ufc_fights.drop(random_indices)

In [25]:
updated_ufc_fights

,date,division,fighter,opponent,result,method,fighter_wins,fighter_losses,fighter_age,fighter_height,...,opponent_abs_leg_strikes_landed_avg,opponent_abs_leg_strikes_attempts_avg,opponent_abs_distance_strikes_landed_avg,opponent_abs_distance_strikes_attempts_avg,opponent_abs_clinch_strikes_landed_avg,opponent_abs_clinch_strikes_attempts_avg,opponent_abs_ground_strikes_landed_avg,opponent_abs_ground_strikes_attempts_avg,fighter_stance,opponent_stance
0,"March 12, 2022",Light Heavyweight,Magomed Ankalaev,Thiago Santos,W,U-DEC,7.0,1.0,29.0,190.5,...,0.631579,0.942275,1.533107,4.268251,0.285229,0.387097,0.458404,0.570458,0,0
2,"March 12, 2022",Bantamweight,Song Yadong,Marlon Moraes,W,KO/TKO,7.0,1.0,24.0,172.72,...,0.523498,0.701963,2.974420,7.816776,0.285544,0.356930,1.213563,1.534801,0,0
4,"March 12, 2022",Featherweight,Sodiq Yusuff,Alex Caceres,W,U-DEC,4.0,1.0,28.0,175.26,...,0.621431,0.796103,2.361438,7.396708,0.201545,0.302318,0.285522,0.473631,0,2
7,"March 12, 2022",Light Heavyweight,Karl Roberson,Khalil Rountree Jr.,L,KO/TKO,4.0,4.0,31.0,185.42,...,0.873728,1.077199,2.956314,6.295631,0.119689,0.191502,0.502693,0.622382,2,2
8,"March 12, 2022",Lightweight,Drew Dober,Terrance McKinney,W,KO/TKO,9.0,7.0,33.0,172.72,...,0.434783,0.434783,0.434783,0.434783,0.000000,0.000000,0.000000,0.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13012,"November 12, 1993",Open Weight,Gerard Gordeau,Kevin Rosier,W,KO/TKO,0.0,0.0,34,195.57999999999998,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
13014,"November 12, 1993",Open Weight,Ken Shamrock,Patrick Smith,W,SUB,0.0,0.0,29,185.42,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
13017,"November 12, 1993",Open Weight,Art Jimmerson,Royce Gracie,L,SUB,0.0,0.0,30,185.42,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2
13018,"November 12, 1993",Open Weight,Kevin Rosier,Zane Frazier,W,KO/TKO,0.0,0.0,unknown,193.04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0


In [26]:
#we worked hard to build this, lets save it to a new file (in case we messed something up... check this)
updated_ufc_fights.to_csv('updated_ufc_fights.csv', index = False)

In [27]:
#we worked hard to build this, lets save it (only run this once we're sure that the new file is correct)
updated_ufc_fights.to_csv('ufc_fights.csv', index = False)

In [ ]:
#add code here to automatically send these to github

In [ ]:
#So the only files we really need are fight_hist,fighter_stats,ufc_fights,ufc_fights_crap, and ufc_fighters